In [2]:
from pybedtools import BedTool

In [1]:
import csv
from collections import defaultdict
from collections import Counter
from glob import glob
import numpy as np
import pandas as pd

In [18]:
from scipy.stats import hypergeom
from statsmodels.stats.multitest import multipletests

### Motifs enrichment 

In [2]:
peaks=pd.read_csv('stage_peaks_anc.tsv',sep='\t')  

In [3]:
peaks

,Peak,Scaf,Start,End,Size,Repeat,Cons,Status,Closest,ClosestN,...,16-cells,EarlyBlastula,LateBlastula,Gastrula,Prism,Pluteus,Pattern,Motif,Motif_TF,cons
0,peak_1,Scaffold_1002,4118,4407,289,0,0,distal,NaN,NaN,...,0,1,1,1,0,0,NYYYNN,"Nuclear_receptor.0005,Homeodomain.0029,Homeodo...","FXR,Gata2,LHX6,ZNF528,DUX4,ZNF274,CUX1,NFAT5,I...",0.000000
1,peak_5,Scaffold_1003,7919,8520,601,0,0,tss,PL04642,unchar-7019,...,1,1,1,1,0,0,YYYYNN,NaN,NaN,0.146463
2,peak_6,Scaffold_1003,8646,8823,177,0,0,genebody,PL04642,unchar-7019,...,0,1,1,0,0,0,NYYNNN,"C2H2_ZF.0109,C2H2_ZF.0236,C2H2_ZF.0264","Ovol1,REST,Ovol1",0.277508
3,peak_15,Scaffold_1006,27294,27555,261,0,0,proximal,PL41561,unchar-21852,...,0,1,1,1,1,0,NYYYYN,"C2H2_ZF.0026,bZIP.0039,bZIP.0043,Mixed.0048,Nu...","ATF3,CEBPB,HLF,SIX5,HNF4G,ETS1,STAT3,PKNOX1,Tc...",0.137238
4,peak_17,Scaffold_1006,29285,30288,1003,0,0,tss,PL41561,unchar-21852,...,1,1,1,1,1,1,YYYYYY,NaN,NaN,0.244763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78469,peak_181278,Scaffold_994,13372,13592,220,0,0,distal,NaN,NaN,...,0,1,1,1,1,0,NYYYYN,"C2H2_ZF.0044,Nuclear_receptor.0051,IRF.0008,Mi...","IKZF1,NR2F2,Irf3,FLI1,CUX1,CUX1,PPARG",0.000000
78470,peak_181279,Scaffold_994,13860,14819,959,0,0,distal,NaN,NaN,...,1,1,1,1,1,0,YYYYYN,"Ets.0002,Homeodomain.0010,bHLH.0010,Sox.0007,D...","ELF5,Rhox8,Olig2,SOX9,DMRT1,Myf5,JUND,IKZF1,MY...",0.031510
78471,peak_181280,Scaffold_994,14959,15062,103,0,0,distal,NaN,NaN,...,0,1,1,0,0,0,NYYNNN,NaN,NaN,0.000000
78472,peak_181281,Scaffold_994,15913,16244,331,0,0,distal,NaN,NaN,...,1,1,1,1,1,0,YYYYYN,"Myb_SANT.0001,Sox.0003,Homeodomain.0049,Homeod...","MYBL1,Sox3,PITX1,Obox1,SOX10,MECOM,HNF4G,STAT1...",0.055393


In [4]:
patDict=pd.Series(peaks.Pattern.values,index=peaks.Peak).to_dict()

In [15]:
#with open('stage_4mael.txt','w') as out:
#    for i, r in peaks[(peaks['Status']!='tss')].iterrows():
#        out.write("{}:{}-{}\t{}\n".format(r['Scaf'], r['Start'], r['End'],r['Pattern']))

In [5]:
fpeaks=peaks[(peaks['Status']!='tss') & (peaks['Repeat']==0) ]

In [21]:
fpeaks

,Peak,Scaf,Start,End,Size,Repeat,Cons,Status,Closest,ClosestN,...,16-cells,EarlyBlastula,LateBlastula,Gastrula,Prism,Pluteus,Pattern,Motif,Motif_TF,cons
0,peak_1,Scaffold_1002,4118,4407,289,0,0,distal,NaN,NaN,...,0,1,1,1,0,0,NYYYNN,"Nuclear_receptor.0005,Homeodomain.0029,Homeodo...","FXR,Gata2,LHX6,ZNF528,DUX4,ZNF274,CUX1,NFAT5,I...",0.000000
2,peak_6,Scaffold_1003,8646,8823,177,0,0,genebody,PL04642,unchar-7019,...,0,1,1,0,0,0,NYYNNN,"C2H2_ZF.0109,C2H2_ZF.0236,C2H2_ZF.0264","Ovol1,REST,Ovol1",0.277508
3,peak_15,Scaffold_1006,27294,27555,261,0,0,proximal,PL41561,unchar-21852,...,0,1,1,1,1,0,NYYYYN,"C2H2_ZF.0026,bZIP.0039,bZIP.0043,Mixed.0048,Nu...","ATF3,CEBPB,HLF,SIX5,HNF4G,ETS1,STAT3,PKNOX1,Tc...",0.137238
6,peak_27,Scaffold_101,1945,2432,487,0,0,distal,NaN,NaN,...,0,1,1,0,0,0,NYYNNN,"bHLH.0007,bHLH.0010,Sox.0006,bHLH.0023,bHLH.00...","Atoh1,Olig2,SOX14,CLOCK,HEY1,HES5,BHLHA15,FOXA...",0.000000
7,peak_53,Scaffold_1012,118456,118679,223,0,0,distal,PL41476,unchar-1347,...,0,1,1,0,0,0,NYYNNN,"Sox.0018,C2H2_ZF.0092,Sox.0024,Mixed.0044,Sox....","SOX10,HIC1,SOX10,BDP1,SOX21,MAF,MEF2A,ZBTB12,F...",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78468,peak_181272,Scaffold_993,4133,5332,1199,0,0,distal,PL41462,NaN,...,1,1,1,1,1,1,YYYYYY,"Homeodomain.0003,MADS_box.0002,bZIP.0011,Myb_S...","HOXB7,AC002126.6,CEBPE,Mybl2,SOX9,Mef2a,KLF9,N...",0.000000
78469,peak_181278,Scaffold_994,13372,13592,220,0,0,distal,NaN,NaN,...,0,1,1,1,1,0,NYYYYN,"C2H2_ZF.0044,Nuclear_receptor.0051,IRF.0008,Mi...","IKZF1,NR2F2,Irf3,FLI1,CUX1,CUX1,PPARG",0.000000
78470,peak_181279,Scaffold_994,13860,14819,959,0,0,distal,NaN,NaN,...,1,1,1,1,1,0,YYYYYN,"Ets.0002,Homeodomain.0010,bHLH.0010,Sox.0007,D...","ELF5,Rhox8,Olig2,SOX9,DMRT1,Myf5,JUND,IKZF1,MY...",0.031510
78471,peak_181280,Scaffold_994,14959,15062,103,0,0,distal,NaN,NaN,...,0,1,1,0,0,0,NYYNNN,NaN,NaN,0.000000


In [6]:
pkTf={}
clGn={}
for i, r in fpeaks.iterrows():
    #print(r['Peak'], r['Closest'],r['Motif_TF'])
    pkTf[r['Peak']]=str(r['Motif_TF']).split(',')
    clGn[r['Peak']]=r['Closest']
    #if i==30: break

### For duplicated genes 

In [23]:
dgenes=pd.read_csv('~/Dropbox/Genomes/Urchin/Paracentrotus_genome/Analyses/Gene_families/dups_classif.tsv',sep='\t')  

In [109]:
dg_new=dgenes[(dgenes['Echinoidea']==1) & (dgenes['Parliv']==0)]['gid']
df_pli=dgenes[(dgenes['Parliv']==1)]['gid']
df_sng=dgenes[(dgenes['Bilateria']==0) & (dgenes['Deuterostomia']==0) & (dgenes['Ambulacraria']==0) & (dgenes['Echinodermata']==0) & (dgenes['Eleutherozoa']==0) & (dgenes['Echinoidea']==0) & (dgenes['Parliv']==0)]['gid']
df_old=dgenes[((dgenes['Bilateria']==1) | (dgenes['Deuterostomia']==1) | (dgenes['Ambulacraria']==1) | (dgenes['Echinodermata']==1) | (dgenes['Eleutherozoa']==1)) & (dgenes['Echinoidea']==0) & (dgenes['Parliv']==0)]['gid']


In [113]:
set(dg_new).intersection(set(df_pli))

set()

In [114]:
gCat={}
for g in set(clGn.values()):
    if g in set(df_pli):
        gCat[g]='plividus'
    elif g in set(dg_new):
        gCat[g]='echinoid'
    elif g in set(df_sng):
        gCat[g]='single'
    elif g in set(df_old):
        gCat[g]='old'
    else:
        gCat[g]='other'

In [115]:
print(Counter(gCat.values()))

Counter({'other': 7139, 'plividus': 3534, 'old': 2948, 'echinoid': 2782, 'single': 2675})


In [ ]:
ctf

### For mfuzz cluster

In [8]:
clfile="/Users/fmarletaz/Dropbox/Genomes/Urchin/Paracentrotus_genome/Analyses/Transcriptomes/mfuzz/Revision/Pliv.mfuzz.clust.txt"

In [10]:
clust={l[0]:'c'+l[1] for l in csv.reader(open(clfile),delimiter='\t')}

In [11]:
clusters=defaultdict(list)
tf_l=[]
for i,p in enumerate(pkTf):
    cl=clust.get(clGn[p],'Other')
    #print(p,pkTf[p],cl)
    clusters[cl].extend(pkTf[p])
    tf_l.extend(pkTf[p])
    #else:
    #oug.extend(pkTf[p])
    

In [12]:
mcl=[]
tf_s=list(set(tf_l))
print(len(tf_s))
for cl in clusters:
    #print(cl)
    clc=Counter(clusters[cl])
    ct_l=[cl]+[clc[tf] for tf in tf_s]
    mcl.append(ct_l)
    #cl_l.append(cl)

684


In [13]:
ctf=pd.DataFrame(mcl, columns=['Cluster']+tf_s).set_index('Cluster').transpose()

In [14]:
ctf

Cluster,Other,c4,c2,c7,c6,c5,c16,c10,c18,c15,c13,c8,c14,c12,c3,c17,c9,c11,c1
FOXQ1,279,51,33,48,20,48,64,40,32,34,36,42,40,22,57,50,55,60,50
ETV6,84,27,11,18,15,13,27,18,18,17,15,25,19,7,30,27,20,18,21
EP300,1019,244,118,160,89,168,242,153,157,213,161,220,172,86,198,214,208,256,177
ZBTB33,797,225,108,139,53,132,211,130,143,183,118,160,127,81,168,158,158,179,151
ZNF76,342,106,47,63,25,52,89,61,68,84,67,92,70,32,110,79,71,105,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NR1H,461,123,55,94,34,67,136,78,96,117,111,119,85,36,99,111,110,122,107
STAT,575,163,80,106,59,107,160,98,88,128,93,134,109,55,115,150,113,133,127
CTCF,3260,825,371,538,269,503,894,503,503,680,583,709,550,306,764,673,729,791,739
Arid3a,328,84,47,40,28,38,85,47,50,67,45,56,42,23,63,45,53,70,70


In [15]:
cl_k=list(clusters.keys())

In [16]:
ctf.to_numpy().sum()

1497391

In [19]:
Hyp=[]
for tf,r in ctf.iterrows():
    phyps=[]
    #print(r)
    for i,v in enumerate(r):
        #print(v,ctf.to_numpy().sum(),ctf[cl_k[i]].sum(),sum(r))
        phyps.append(hypergeom.sf(v,ctf.to_numpy().sum(),ctf[cl_k[i]].sum(),sum(r)))
    #print(tf,i,phyps)
    Hyp.append([tf]+phyps)
        #Hyp[tf]=phyp
        #if phyp<0.01:
        #    Hyp.append([tf,r[0],r[1],phyp])
    #break

In [20]:
resd = pd.DataFrame(Hyp, columns = ['TF']+cl_k).set_index('TF')

In [ ]:
resd[resd['c18']<0.01]

In [23]:
cpv=[]
for c in  list(resd):
    pvals=multipletests(resd[c], method='fdr_bh')[1]
    cpv.append([c]+list(pvals))

In [24]:
resm=pd.DataFrame(cpv, columns = ['Cluster']+tf_s).set_index('Cluster').transpose()

In [ ]:
resm[resm['c18']<0.05]

In [27]:
resm.to_csv('enrich_motifs_mfuzz18.tsv',sep='\t')

In [312]:
padjPat.to_csv('enrich_motifs.tsv',sep='\t')

In [298]:
padjPat.sum()

In [302]:
max(p_adjusted[1])

1.0

In [297]:
from statsmodels.sandbox.stats.multicomp import multipletests